---
# --- Day 7: No Space Left On Device ---
---

In [15]:
from typing import List
import numpy as np

## Load data

In [16]:
full_puzzle_data = True

In [17]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day07_input{file_suffix}.txt", "r") as f:
    data = f.read().splitlines()

In [18]:
class DirectoryNode:
    def __init__(self, name, parent_dir=None):
        self.name = name
        self.parent_directory = parent_dir
        self.file_size = 0
        self.subdirectories = {}
        
    def __str__(self) -> str:
        return f"DirectoryNode<{self.name}>"
    
    def create_subdirectory(self, dir_name: str):
        subdir_name = self.name + dir_name + "/"
        self.subdirectories[dir_name] = DirectoryNode(subdir_name, self)        
    
    def update_file_size(self, file_size: int):
        self.file_size += file_size
    
    def go_to_subdirectory(self, dir_name: str):
        return self.subdirectories[dir_name]
    
    def move_to_parent(self):
        return self.parent_directory

In [19]:
def populate_file_system(instructions: List[str]):
    if instructions[0] != "$ cd /":
        return None
    root_node = DirectoryNode("/")
    this_node = root_node
    i = 1
    while (i < len(instructions)):
        comm = instructions[i][2:5]
        if comm == "ls":
            i += 1
            while instructions[i][0] != "$":                
                ins = instructions[i]
                if ins[:3] == "dir":
                    this_node.create_subdirectory(ins.split()[1])
                else:
                    this_node.update_file_size(int(ins.split()[0]))
                i += 1
                if i == len(instructions):
                    break
        else: # cd command
            target_dir = instructions[i][5:]
            if target_dir == "..":
                this_node = this_node.move_to_parent()
            else:
                this_node = this_node.go_to_subdirectory(target_dir)
            i += 1
    return root_node

In [20]:
root_node = populate_file_system(data)

## --- Part One ---

In [21]:
def gather_all_directories(node):
    all_nodes = {node.name: node}
    for i, subnode in node.subdirectories.items():
        all_nodes.update(gather_all_directories(subnode))
    return all_nodes

In [22]:
def get_size(node):
    total_size = node.file_size
    for i, subnode in node.subdirectories.items():
        total_size += get_size(subnode)
    return total_size

In [23]:
all_dirs = gather_all_directories(root_node)

In [24]:
dir_sizes = {k:get_size(node) for k, node in all_dirs.items()}

In [25]:
max_size = 100000
sizesum = sum([v for _, v in dir_sizes.items() if v <= 100000])
print(f"The sum of the total sizes of the 'small directories' is {sizesum}.")

The sum of the total sizes of the 'small directories' is 1084134.


## --- Part Two ---

In [26]:
space_to_free = dir_sizes["/"] - 40000000

In [27]:
s = np.array([v for _, v in dir_sizes.items()])
s.sort()

In [28]:
smallest_size = s[np.where(s >= space_to_free)[0][0]]
print(f"The size of the smallest directory to be deleted is {smallest_size}.")

The size of the smallest directory to be deleted is 6183184.
